# К промежуточному дедлайну было подготовлено:

Был запарсен сайт https://bolshayastrana.com/tury, итоговые данные были сохарены в таблицу pandas (прикреплена отдельным файлом). 

Описание Данный проект занимается сбором данных о туристических предложениях по внутреннему туризму в Российской Федерации с портала "Большая страна". Цель проекта - собрать актуальную информацию о турах для последующего анализа и использования в туристических и маркетинговых целях.

Методика сбора данных Для сбора данных используется Selenium WebDriver, который позволяет автоматизировать взаимодействие с веб-браузером. Скрипт проходит по страницам сайта, извлекает необходимую информацию о каждом туре и сохраняет ее в структурированном виде.

Собираемые данные Для каждого тура собираются следующие данные:

Название тура Цена Локация (регион) Даты проведения (даты начала и окончания тура) Рейтинг тура Количество оценок Сложность тура (выраженная как доля от максимально возможной) Уровень комфорта Прямая ссылка на тур Тэги, ассоциированные с туром Длительность тура Тип проживания во время тура Размер группы Минимальный возраст участников Место встречи участников Примеры использования данных Собранные данные могут быть использованы для анализа предпочтений клиентов, планирования маркетинговых кампаний, исследования рынка внутреннего туризма, а также для разработки новых туристических продуктов.

Инструкции по запуску скрипта Установите необходимые библиотеки: selenium, pandas. Загрузите драйвер для вашего браузера (например, ChromeDriver для Google Chrome). Запустите скрипт для сбора данных. Данные будут сохранены в файл data_tours.csv. Лицензия Укажите тип лицензии, под которой распространяется ваш код (например, MIT License).

Это описание подчеркивает практическую пользу собранных данных и объясняет, как можно использовать проект для реальных задач в туризме и маркетинге. Помимо технической документации, также важно упомянуть, как именно данные могут быть применены на практике, что повысит интерес потенциальных пользователей проекта.

Код парсинга: (запускали в PyCharm)

In [ ]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from time import sleep

driver = webdriver.Chrome()
page_driver = webdriver.Chrome()
data_dict = {}
for i in range(1, 2):
    url = 'https://bolshayastrana.com/tury?plainSearch=1&page=' + str(i)
    try:
        driver.get(url)
        tours_block = driver.find_element(By.CLASS_NAME, "tour-previews")
        tours = tours_block.find_elements(By.CLASS_NAME, "as-col")
        for j, tour in enumerate(tours):
            tour_dict = {}
            try:
                tour_preview = tour.find_element(By.CLASS_NAME, "tour-preview")
                location = tour_preview.find_element(By.CSS_SELECTOR, "[itemprop='location']").find_element(
                    By.CSS_SELECTOR,
                    "meta[itemprop='addressRegion']")
                offers = tour_preview.find_elements(By.CSS_SELECTOR, "[itemprop='offers']")[:-1]
                prices = [offer.find_element(By.CSS_SELECTOR, "[itemprop='price']").get_attribute("content") for offer
                          in offers]
                dates = [(offer.find_element(By.CSS_SELECTOR, "meta[itemprop='availabilityStarts']").get_attribute(
                    'content'), offer.find_element(By.CSS_SELECTOR, "meta[itemprop='availabilityEnds']").get_attribute(
                    'content')) for offer in offers]
                ratings = tour_preview.find_element(By.CSS_SELECTOR, "[itemprop='aggregateRating']")
                rating = ratings.find_element(By.CSS_SELECTOR, "meta[itemprop='ratingValue']").get_attribute("content")
                rating_counts = ratings.find_element(By.CSS_SELECTOR, "meta[itemprop='ratingCount']").get_attribute(
                    "content")
                main_block = tour_preview.find_element(By.CLASS_NAME, "tour-preview__content")
                title = main_block.find_element(By.CLASS_NAME, "tour-preview__title").text
                difficulty = len(
                    main_block.find_element(By.CLASS_NAME, "tour-preview__footer").find_element(By.CLASS_NAME,
                                                                                                "tour-preview__footer-difficulty").find_element(
                        By.CLASS_NAME, "difficulty-dots").find_element(By.CLASS_NAME,
                                                                       "difficulty-dots__rating").find_elements(
                        By.CSS_SELECTOR, ".difficulty-dotsitem.difficulty-dotsitem--active"))
                comfort = main_block.find_element(By.CLASS_NAME, "tour-preview__footer").find_element(By.CLASS_NAME,
                                                                                                      "tour-preview__footer-accommodation").find_element(
                    By.CLASS_NAME, "tour-preview__accommodation").find_element(By.CLASS_NAME,
                                                                               "accommodation-comfort").text
                cur_url = main_block.find_element(By.CLASS_NAME, "tour-preview__title").get_attribute("href")
                page_driver.get(cur_url)
                tour_content_section = page_driver.find_element(By.CLASS_NAME, "tour-content__section")
                description = tour_content_section.find_element(By.CLASS_NAME, "article-text").text
                tags = tour_content_section.find_element(By.CLASS_NAME, "tour-main-info__tags").text.split('\n')
                sidebar_card = page_driver.find_element(By.CLASS_NAME, "sidebar-card")
                duration = sidebar_card.find_element(By.CLASS_NAME, "as-flex").text.split('\n')
                living_place = sidebar_card.find_element(By.CLASS_NAME, "accommodation").text.split('\n')
                sidebar_card_body = sidebar_card.find_elements(By.CLASS_NAME, "sidebar-card-body")[3]
                sidebar_card_body_text = sidebar_card_body.text.split('\n')
                group_size = sidebar_card_body_text[1]
                min_age = sidebar_card_body_text[3]
                meeting_place = sidebar_card_body_text[5]

                print(f"Title: {title}")
                print(f"Location: {location.get_attribute('content')}")
                print(f"Dates: {dates[0]}")
                print(f"Prices: {prices[0]}")
                print(f"Rating: {rating}")
                print(f"Rating count: {rating_counts}")
                print(f"Difficulty: {difficulty}/4")
                print(f"Comfort: {comfort}")
                print(f"Url: {cur_url}")
                print(f"Description: {description}")
                print(f"Tags: {tags}")
                print(f"Duration: {duration[0]}")
                print(f"Living place: {living_place[0]}")
                print(f"Group size: {group_size}")
                print(f"Min age: {min_age}")
                print(f"Meeting place: {meeting_place}")
                print()
                tour_dict['Название тура'] = title
                tour_dict['Цена'] = prices[0]
                tour_dict['Локация'] = location.get_attribute('content')
                tour_dict['Приб/отб'] = list(dates[0])
                tour_dict['Рейтинг'] = rating
                tour_dict['Кол-во оценок'] = rating_counts
                tour_dict['Сложность'] = difficulty/4
                tour_dict['Комфорт'] = comfort
                tour_dict['Url'] = cur_url
                tour_dict['Тэги'] = tags
                tour_dict['Длительность'] = duration[0]
                tour_dict['Место прожив.'] = living_place[0]
                tour_dict['Размер группы'] = group_size
                tour_dict['Мин. возраст'] = min_age
                tour_dict['Место встречи'] = meeting_place
                data_dict[f'{i}_{j}'] = tour_dict

            except:
                pass

    except:
        pass
df = pd.DataFrame.from_dict(data_dict, orient = 'index')
df.to_csv('data_tours.csv')

Также были выдвинуты три гипотезы:
1.	Гипотеза: Туры у которых ниже минимальный возрастной порог будут стоит дороже туров, где он выше
•	Модель: Линейная регрессия 
2.	Гипотеза: Цена тура будет коррелировать с уровнем комфорта и сложностью тура.
•	Модель: Кластеризация (например, K-means) для определения ценовых сегментов и их корреляции с другими характеристиками, такими как рейтинг, сложность, продолжительность и комфорт.
3.	Гипотеза: Рейтинги туров могут быть предсказаны на основе их характеристик: цену, комфорт, сложность
Модель машинного обучения: Регрессия (например, линейная регрессия или градиентный бустинг). Также будут проводиться тесты на значимость

